In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/pneumonia-xray-images'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#!pip install -q wandb

In [ ]:
#import wandb
#wandb.login()

In [ ]:
#wandb.init('simclr_101')

In [ ]:
#best_model = wandb.restore('resnet50_simclr.h5', run_path="gowthambg/simclr_101/wise-wind-4")

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from PIL import Image
image = Image.open('../input/pediatric-pneumonia-chest-xray/Pediatric Chest X-ray Pneumonia/train/NORMAL/IM-0129-0001.jpeg')
print(image.size)

In [ ]:
import pandas as pd
dataset = pd.read_csv("../input/sample/sample/sample_labels.csv")

In [ ]:
labels = []
for i in dataset["Finding Labels"].str.split("|"):
    for j in i:
        labels.append(j)

In [ ]:
uniq_labels = list(set(labels))

In [ ]:
dataset.columns

In [ ]:
uniq_labels

In [ ]:
def preproc(df):
    filename = []
    data = { val:[] for val in uniq_labels}
    for row in df[["Image Index","Finding Labels"]].to_numpy():
        filename.append(row[0])
        row_labels = row[1].split("|")
        for label in uniq_labels:
            data[label].append(1) if label in row_labels else data[label].append(0)
    data["filename"]=filename
    return pd.DataFrame(data)

In [ ]:
new_dataset = preproc(dataset)

In [ ]:
train = new_dataset[:int(len(new_dataset)*0.9)]
test = new_dataset[int(len(new_dataset)*0.9)+1:]

In [ ]:
train.head()

In [ ]:
print(train.shape, test.shape)

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
# training_set = train_datagen.flow_from_directory(
#         '../input/pediatric-pneumonia-chest-xray/Pediatric Chest X-ray Pneumonia/train',
#         target_size=(224, 224),
#         batch_size=32,
#         class_mode='binary')
training_set = train_datagen.flow_from_directory(
        directory="../input/pneumonia-xray-images/train",
        batch_size=32,
        seed=42,
        shuffle=True,
        #class_mode="raw",
        target_size=(32,32)
)

In [ ]:
import os
test["filename"].iloc[0] in os.listdir("../input/sample/sample/sample/images/")


In [ ]:
test.head()

In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
# test_set = test_datagen.flow_from_dataframe(
#         dataframe=test,
#         directory="../input/sample/sample/sample/images/",
#         x_col="filename",
#         y_col=uniq_labels,
#         subset="validation",
#         target_size=(224, 224),
#         batch_size=16,
#         shuffle=True,
#         class_mode="raw"
# )
test_set = test_datagen.flow_from_directory(
        #dataframe=test,
        directory="../input/pneumonia-xray-images/val",
        #x_col="filename",
        #y_col=uniq_labels,
        #subset="training",
        batch_size=32,
        seed=42,
        shuffle=True,
        #class_mode="raw",
        target_size=(32,32)
)

In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=64,activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=2,activation='sigmoid'))

In [ ]:
from keras.models import model_from_json
json_file = open('../input/densenet121-model/den121_simclr.json', 'r')
loaded_model_json = json_file.read()
json_file.close()


In [ ]:
cnn = model_from_json(loaded_model_json)


In [ ]:
cnn.summary()

In [ ]:
cnn.load_weights('../input/densenet121-model/densenet121_simclr2.h5')

In [ ]:
cnn.non_trainable_weights

In [ ]:
ResNet = tf.keras.models.Sequential()
ResNet.add(cnn)

In [ ]:
ResNet.summary()

In [ ]:
ResNet.add(tf.keras.layers.Dense(units=2, activation='sigmoid'))


In [ ]:
ResNet.summary()

In [ ]:
def step_decay(epoch):
   initial_lrate = 0.003
   drop = 0.5
   epochs_drop = 2.0
   lrate = initial_lrate * math.pow(drop,  
           math.floor((1+epoch)/epochs_drop))
   return lrate
class LearningRate(tf.keras.callbacks.Callback):
  
  def on_train_begin(self,logs={}):
    self.lr_epoch=[]

  def on_epoch_end(self, batch, logs={}):
    self.lr_epoch.append(step_decay(len(self.lr_epoch)+1))

In [ ]:
def step_decay(epoch):
   initial_lrate = 0.003
   drop = 0.5
   epochs_drop = 2.0
   lrate = initial_lrate * pow(drop,  
           (1+epoch)//epochs_drop)
   return lrate


epochs=10

lr_history=LearningRate()
lrate = tf.keras.callbacks.LearningRateScheduler(step_decay)

In [ ]:
ResNet.compile(optimizer=tf.keras.optimizers.Adam(lr=3*1e-5),loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])
#ResNet.compile(optimizer=tf.keras.optimizers.RMSprop(lr=|0.0001),loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

In [ ]:
res = ResNet.fit(x = training_set,validation_data=test_set,epochs=20)

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(res.history['accuracy'])
plt.plot(res.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.figure()
plt.plot(res.history['loss'])
plt.plot(res.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
import matplotlib.pyplot as plt
# from imutils import paths
from tqdm import tqdm
import tensorflow as tf
import numpy as np
import cv2


In [ ]:
base_model = tf.keras.applications.InceptionV3(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
base_model.trainable = True
inputs = Input((224, 224, 3))
h = base_model(inputs, training=True)
h = GlobalAveragePooling2D()(h)

projection_1 = Dense(1024)(h)
projection_1 = Activation("relu")(projection_1)
projection_2 = Dense(512)(projection_1)
projection_2 = Activation("relu")(projection_2)
# projection_3 = Dense(50,activation='relu')(projection_2)
projection_4 = Dense(1,activation='sigmoid')(projection_2)

resnet = Model(inputs, projection_4)

In [ ]:
resnet.summary()

In [ ]:
# resnet.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),loss='binary_crossentropy',metrics=['accuracy'])
resnet.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),loss='binary_crossentropy',metrics=['accuracy','mae'])

In [ ]:
result = resnet.fit(x = training_set,validation_data=test_set,epochs=50)

In [ ]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet import ResNet101
#from keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [ ]:
IMAGE_SIZE = [224, 224]
resnet = ResNet101(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:

# don't train existing weights
for layer in resnet.layers:
    layer.trainable = False

In [ ]:
# useful for getting number of output classes
folders = glob('../input/pneumonia-xray-images/train/*')

In [ ]:
# our layers - you can add more if you want
x = Flatten()(resnet.output)

In [ ]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=resnet.input, outputs=prediction)

In [ ]:
model.summary()

In [ ]:
import tensorflow as tf
#model.compile(
#  loss='categorical_crossentropy',
#  optimizer='adam',
 # metrics=['accuracy']
#)
model.compile(optimizer=tf.keras.optimizers.Adamax(lr=0.1),loss='binary_crossentropy',metrics=['accuracy',tf.keras.metrics.AUC(),tf.keras.metrics.Precision()])

In [ ]:

# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
# Make sure you provide the same target size as initialied for the image size
training_set2 = train_datagen.flow_from_directory('../input/pneumonia-xray-images/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

In [ ]:
test_set2 = test_datagen.flow_from_directory('../input/pneumonia-xray-images/val',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

In [ ]:
r = model.fit_generator(
  training_set2,
  validation_data=test_set2,
  epochs=20
)